# Обязательная часть
Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>
В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

# Дополнительная часть (необязательная)
Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [1]:
# imports
import requests as req
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
from datetime import datetime

In [11]:
def scrap_habr(search_str, page=1):
    """
    :param
      search_str: поисковый запрос array
      page: пагинация

    :return DF Pandas
    """

    scrap_df = pd.DataFrame()
    params_req = {
        'url': 'https://habr.com/ru/search/',
        'search': {
            'target_type': 'posts',
            'order': 'relevance',
        }}
    for search in search_str:
        for i in range(1,page+1):
            print(f'Ищем {search} на странице', i)
            params_req['search']['q'] = search
            try:
                resp = req.get(params_req['url'] + f'page{i}/', params=params_req['search'])
                to_data = bs(resp.text)
                data = to_data.find_all('article', class_='tm-articles-list__item')
                for item in data:
                    date = datetime.strptime(item.find('time')['title'],'%Y-%m-%d, %H:%M')
                    title_pre = item.find('h2', class_='tm-article-snippet__title') or item.find('h2', class_='tm-megapost-snippet__title')
                    title = title_pre.text
                    pre_link = item.find('a', class_='tm-article-snippet__title-link') or item.find('a', class_='tm-megapost-snippet__card')
                    link = 'https://habr.com' + pre_link.get('href')
                    likes = item.find('span', class_='tm-votes-meter__value_rating').text
                    try:
                        req_post = req.get(link) # t-records
                        post = bs(req_post.text)
                        prep_post = post.find('div', class_='article-formatted-body') or post.find('div', class_='t-records')
                        full_text = prep_post.text.strip()
                        row = {
                            'date': date,
                            'title': title,
                            'link': link,
                            'likes': likes,
                            'full_text': full_text,
                        }
                        scrap_df = pd.concat([scrap_df, pd.DataFrame([row])])
                        time.sleep(0.1)
                        print(f'parse post "{title}" done')
                    except NameError:
                        print(f'Не удалось распарсить статью: {title} (link: {link})')
                        print(NameError)
                time.sleep(0.2)
            except NameError:
                print('Something wrong', NameError)
            print(f'page {i} scrap done')
    return scrap_df.reset_index(drop=True)

df = scrap_habr(['анализ данных'], 1)
df


Ищем анализ данных на странице 1
parse post "Пять лет Школе анализа данных" done
parse post "10 лет Школе анализа данных Яндекса" done
parse post "Что такое бессерверный SQL? И как использовать его для анализа данных?" done
parse post "Как мы строим систему обработки, хранения и анализа данных в СИБУРе" done
parse post "Ubic создаст платформу для анализа данных о москвичах" done
parse post "Microsoft анонсировала запуск «Планетарного компьютера» для сбора, хранения и анализа данных о состоянии Земли" done
parse post "Владимир Путин выступил с Германом Грефом на конференции по ИИ и анализу данных Artificial Intelligence Journey 2021" done
parse post "Интенсив для повышения квалификации: как использовать Python для анализа данных" done
parse post "AI Journey Contest 2021: какие задачи мы подготовили для участников нового чемпионата по анализу данных" done
parse post "Обзор наиболее интересных материалов по анализу данных и машинному обучению №1 (9 — 16 июня 2014)" done
parse post "Обзор 

,date,title,link,likes,full_text
0,2012-07-25 13:29:00,Пять лет Школе анализа данных,https://habr.com/ru/company/yandex/blog/148443/,+21,Ровно пять лет назад Яндекс объявил об открыти...
1,2017-07-25 13:47:00,10 лет Школе анализа данных Яндекса,https://habr.com/ru/company/yandex/blog/334066/,+53,Сегодня исполняется 10 лет Школе анализа данны...
2,2021-01-14 10:00:00,Что такое бессерверный SQL? И как использовать...,https://habr.com/ru/company/microsoft/blog/537...,+2,Каждому специалисту по обработке и анализу дан...
3,2019-01-21 11:21:00,"Как мы строим систему обработки, хранения и ан...",https://habr.com/ru/company/sibur_official/blo...,+16,В начале 2018 года у нас активно пошел процесс...
4,2020-05-14 13:39:00,Ubic создаст платформу для анализа данных о мо...,https://habr.com/ru/news/t/501944/,+14,Правительство Москвы планирует создать общегор...
5,2020-04-16 07:10:00,Microsoft анонсировала запуск «Планетарного ко...,https://habr.com/ru/news/t/497474/,+13,В рамках развития программы Microsoft AI for E...
6,2021-11-13 19:15:00,Владимир Путин выступил с Германом Грефом на к...,https://habr.com/ru/news/t/588991/,+8,12 ноября президент России вместе с главой «Сб...
7,2021-12-10 13:15:00,Интенсив для повышения квалификации: как испол...,https://habr.com/ru/company/netologyru/news/t/...,+4,"14 декабря в Нетологии пройдёт вебинар, посвящ..."
8,2021-10-18 15:08:00,AI Journey Contest 2021: какие задачи мы подго...,https://habr.com/ru/company/sbercloud/news/t/5...,+8,Нарисовать карту лесных пожаров и распознать п...
9,2014-06-18 15:19:00,Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/226641/,+19,Данный выпуск дайджеста наиболее интересных ма...
